In [1]:
# --- 1. Load environment variables ---
from dotenv import load_dotenv
import os
from pathlib import Path

# Look for .env in repo root (adjust if it's in another dir)
env_path = Path("..") / ".env"
if not env_path.exists():
    raise FileNotFoundError(f".env not found at {env_path.resolve()}")

load_dotenv(env_path)

# --- 2. Read values ---
PROJECT_ID = os.getenv("PROJECT_ID")
DATASET_NAME = os.getenv("DATASET_NAME")
LOCATION = os.getenv("LOCATION", "us-central1")
SA_KEY  = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")

print("✅ Loaded env variables:")
print("PROJECT_ID:", PROJECT_ID)
print("DATASET_NAME:", DATASET_NAME)
print("LOCATION:", LOCATION)


✅ Loaded env variables:
PROJECT_ID: abstract-arc-469501-e2
DATASET_NAME: bigquery_comp_dataset
LOCATION: US


In [2]:
# --- 3. Check Google Cloud authentication ---
from google.oauth2 import service_account

if not SA_KEY or not Path(SA_KEY).exists():
    raise FileNotFoundError("Service account key file not found. Check GOOGLE_APPLICATION_CREDENTIALS.")

creds = service_account.Credentials.from_service_account_file(SA_KEY)

print("✅ Authenticated with service account:", creds.service_account_email)


✅ Authenticated with service account: 597564348526-compute@developer.gserviceaccount.com


In [3]:
# --- Cell 3: Test BigQuery client ---
from google.cloud import bigquery

bq_client = bigquery.Client(project=PROJECT_ID, credentials=creds)

# Quick test query
query = "SELECT 'Hello from BigQuery' as msg"
rows = list(bq_client.query(query).result())
print(rows[0].msg)


Hello from BigQuery


In [4]:
# --- Cell 4: Test Vertex AI Search client (Discovery Engine) ---
from google.cloud import discoveryengine_v1 as de

# A SearchServiceClient can list or query once you have an Engine
search_client = de.SearchServiceClient(credentials=creds)

# Just check the client instantiates without error
print("✅ Vertex AI Search client is ready.")

# If you want to confirm resources exist, you can list DataStores in the default collection
ds_client = de.DataStoreServiceClient(credentials=creds)
parent = f"projects/{PROJECT_ID}/locations/global/collections/default_collection"

for ds in ds_client.list_data_stores(parent=parent):
    print(" - DataStore:", ds.name)


✅ Vertex AI Search client is ready.


In [5]:
# --- Cell 5: Confirm GCS bucket access ---
from google.cloud import storage

storage_client = storage.Client(project=PROJECT_ID, credentials=creds)
buckets = list(storage_client.list_buckets(max_results=3))

print("✅ GCS connected. Sample buckets:")
for b in buckets:
    print(" -", b.name)


✅ GCS connected. Sample buckets:
 - 597564348526_us_import
 - cloud-ai-platform-3494133b-91fb-4ec3-b136-8334901360ba
 - cloud-ai-platform-523554d0-0c5c-493c-a958-a89dbbab84fc
